In [3]:
import logging
import os
from fetch_data import (
    fetch_thl_cases_erva_daily, construct_cases_age_erva_daily,
    static_population_erva_age, construct_finland_age_cases_daily,
    fetch_hs_hospitalizations,
)
import pandas as pd
import numpy as np

curr_dir = os.getcwd()
stats_dir = os.path.join(curr_dir, 'stats')
logger = logging.getLogger(__name__)
curr_dir

'/Users/aponcedeleonch/Documents/Aalto/Courses/MasterThesis/Code/jeta'

In [2]:
age_file = os.path.join(stats_dir, 'erva_population_age_2020.csv')
pop_age, pop_age_prop = static_population_erva_age(logger, age_file)
pop_age

,erva,age_group,Total
0,All,0-9,547942
1,All,10-19,611151
2,All,20-29,661400
3,All,30-39,718761
4,All,40-49,666421
...,...,...,...
58,Åland,40-49,3853
59,Åland,50-59,4158
60,Åland,60-69,3896
61,Åland,70-79,3243


In [3]:
erva_cases = fetch_thl_cases_erva_daily(logger)

In [4]:
erva_cases.loc[erva_cases['Time'] >= '2021-01-04', ]

,Time,erva,val
2584,2021-01-04,HYKS,225.0
2585,2021-01-04,KYS,12.0
2586,2021-01-04,OYS,37.0
2587,2021-01-04,TAYS,38.0
2588,2021-01-04,TYKS,82.0
...,...,...,...
5112,2021-12-31,KYS,0.0
5113,2021-12-31,OYS,0.0
5114,2021-12-31,TAYS,0.0
5115,2021-12-31,TYKS,0.0


In [5]:
age, age_prop = construct_finland_age_cases_daily(logger)

In [6]:
age.loc[age['Time'] >= '2021-01-04', ]

,Time,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
371,2021-01-04,12.857143,28.428571,65.714286,50.714286,40.714286,30.857143,17.0,8.857143,6.857143
372,2021-01-05,12.857143,28.428571,65.714286,50.714286,40.714286,30.857143,17.0,8.857143,6.857143
373,2021-01-06,12.857143,28.428571,65.714286,50.714286,40.714286,30.857143,17.0,8.857143,6.857143
374,2021-01-07,12.857143,28.428571,65.714286,50.714286,40.714286,30.857143,17.0,8.857143,6.857143
375,2021-01-08,12.857143,28.428571,65.714286,50.714286,40.714286,30.857143,17.0,8.857143,6.857143
...,...,...,...,...,...,...,...,...,...,...
730,2021-12-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
731,2021-12-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
732,2021-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000
733,2022-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000


In [7]:
age_prop.loc[age_prop['Time'] >= '2021-01-04', ]

,Time,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
371,2021-01-04,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
372,2021-01-05,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
373,2021-01-06,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
374,2021-01-07,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
375,2021-01-08,0.049073,0.108506,0.250818,0.193566,0.155398,0.117775,0.064885,0.033806,0.026172
...,...,...,...,...,...,...,...,...,...,...
730,2021-12-29,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
731,2021-12-30,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
732,2021-12-31,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
733,2022-01-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [8]:
age_days = construct_cases_age_erva_daily(logger)

In [9]:
age_days.loc[age_days['Time'] >= '2021-01-04', ]

,Time,erva,0-9,10-19,20-29,30-39,40-49,50-59,60-69,70-79,80+
2214,2021-01-04,HYKS,11.041439,24.413850,56.434024,43.552345,34.964558,26.499455,14.599237,7.606325,5.888768
2215,2021-01-04,KYS,0.588877,1.302072,3.009815,2.322792,1.864776,1.413304,0.778626,0.405671,0.314068
2216,2021-01-04,OYS,1.815703,4.014722,9.280262,7.161941,5.749727,4.357688,2.400763,1.250818,0.968375
2217,2021-01-04,TAYS,1.864776,4.123228,9.531080,7.355507,5.905125,4.475463,2.465649,1.284624,0.994547
2218,2021-01-04,TYKS,4.023991,8.897492,20.567067,15.872410,12.742639,9.657579,5.320611,2.772083,2.146129
...,...,...,...,...,...,...,...,...,...,...,...
4381,2021-12-31,KYS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4382,2021-12-31,OYS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4383,2021-12-31,TAYS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4384,2021-12-31,TYKS,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [10]:
comp = compartment_values_daily(logger, age_file)

In [11]:
comp.loc[comp['date'] >= '2021-01-04', ]

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered
369,2021-01-04,HYKS,0-9,207567.673647,69.976068,699.760684,769.736752,657.092879,12618.496722
370,2021-01-05,HYKS,0-9,207475.62692,64.133747,641.337472,705.471219,627.683772,12804.218089
371,2021-01-06,HYKS,0-9,207404.876654,59.735716,597.357163,657.092879,608.826622,12942.203844
372,2021-01-07,HYKS,0-9,207309.544114,57.062161,570.621611,627.683772,638.842914,13036.929201
373,2021-01-08,HYKS,0-9,207244.438291,55.347875,553.478747,608.826622,605.704462,13154.030625
...,...,...,...,...,...,...,...,...,...
726,2021-12-27,Åland,80+,1698.50724,0.0,0.0,0.0,0.0,13.49276
727,2021-12-28,Åland,80+,1698.50724,0.0,0.0,0.0,0.0,13.49276
728,2021-12-29,Åland,80+,1698.50724,0.0,0.0,0.0,0.0,13.49276
729,2021-12-30,Åland,80+,1698.50724,0.0,0.0,0.0,0,13.49276


In [12]:
epidemic_state = full_epidemic_state_finland(logger, age_file)

In [13]:
epidemic_state.loc[epidemic_state['date'] >= '2021-01-04', ]

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered,First dose,Second dose,First dose cumulative,Second dose cumulative
369,2021-01-04,HYKS,0-9,207567.673647,69.976068,699.760684,769.736752,657.092879,12618.496722,0,0,0,0
370,2021-01-05,HYKS,0-9,207475.626920,64.133747,641.337472,705.471219,627.683772,12804.218089,0,0,0,0
371,2021-01-06,HYKS,0-9,207404.876654,59.735716,597.357163,657.092879,608.826622,12942.203844,0,0,0,0
372,2021-01-07,HYKS,0-9,207309.544114,57.062161,570.621611,627.683772,638.842914,13036.929201,0,0,0,0
373,2021-01-08,HYKS,0-9,207244.438291,55.347875,553.478747,608.826622,605.704462,13154.030625,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
39469,2021-12-27,Åland,80+,165.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1533,266
39470,2021-12-28,Åland,80+,165.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1533,266
39471,2021-12-29,Åland,80+,165.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1533,266
39472,2021-12-30,Åland,80+,165.507240,0.000000,0.000000,0.000000,0.000000,13.492760,0,0,1533,266


In [4]:
hosp_by_erva = fetch_hs_hospitalizations(logger)

In [5]:
hosp_by_erva

,date,erva,hospitalized,ward,icu,dead
0,2020-03-21,HYKS,0,0,0,1
2,2020-03-25,HYKS,63,45,18,2
3,2020-03-25,KYS,6,3,3,0
4,2020-03-25,OYS,4,3,1,0
5,2020-03-25,TAYS,7,7,0,1
...,...,...,...,...,...,...
1475,2021-04-16,HYKS,100,79,21,575
1476,2021-04-16,KYS,11,6,5,81
1477,2021-04-16,OYS,10,10,1,55
1478,2021-04-16,TAYS,19,17,2,79


In [25]:
ep8 = pd.read_csv('out/epidemic_finland_8.csv')

In [31]:
ep8_zero = ep8.loc[ep8['date'] == '2021-04-19', :]

In [32]:
ep8_zero

,date,erva,age,susceptible,infected detected,infected undetected,infected,exposed,recovered,First dose,Second dose,First dose cumulative,Second dose cumulative
474,2021-04-19,HYKS,0-9,171968.029333,96.835420,968.354204,1065.189624,153.035778,48426.745264,0.0,0.0,0.0,0.0
1205,2021-04-19,HYKS,10-19,210510.226415,111.297853,293.349876,404.647729,58.135733,25862.990123,0.0,0.0,1477.0,252.0
1936,2021-04-19,HYKS,20-29,230974.806567,123.873882,198.605429,322.479310,46.330598,29230.383525,0.0,0.0,12100.0,3556.0
2667,2021-04-19,HYKS,30-39,274604.745607,123.245081,159.884157,283.129238,40.677174,22519.447981,0.0,0.0,18725.0,5522.0
3398,2021-04-19,HYKS,40-49,240229.218303,113.184258,132.618263,245.802521,35.314445,17442.664731,0.0,0.0,28035.0,6167.0
4129,2021-04-19,HYKS,50-59,227751.874175,72.940966,80.938608,153.879574,22.107876,13796.138375,0.0,0.0,47404.0,7020.0
4860,2021-04-19,HYKS,60-69,153783.865135,32.068873,34.475368,66.544241,9.560410,6747.030213,0.0,0.0,95399.0,4039.0
5591,2021-04-19,HYKS,70+,40662.565066,29.553667,31.150475,60.704142,8.721363,5875.009428,0.0,0.0,271680.0,18908.0
6322,2021-04-19,KYS,0-9,144220.362234,6.887299,68.872987,75.760286,9.091234,4537.786246,0.0,0.0,0.0,0.0
7053,2021-04-19,KYS,10-19,116541.544767,7.915921,20.864145,28.780066,3.453608,2706.221559,0.0,0.0,443.0,122.0


In [33]:
ep8_zero = ep8_zero[~ep8_zero['erva'].str.contains('land')]
ep8_zero = ep8_zero.drop(columns=['date', 'erva', 'age', 'First dose', 'Second dose', 'Second dose cumulative', 'infected detected', 'infected undetected'])
ep8_zero

,susceptible,infected,exposed,recovered,First dose cumulative
474,171968.029333,1065.189624,153.035778,48426.745264,0.0
1205,210510.226415,404.647729,58.135733,25862.990123,1477.0
1936,230974.806567,322.479310,46.330598,29230.383525,12100.0
2667,274604.745607,283.129238,40.677174,22519.447981,18725.0
3398,240229.218303,245.802521,35.314445,17442.664731,28035.0
4129,227751.874175,153.879574,22.107876,13796.138375,47404.0
4860,153783.865135,66.544241,9.560410,6747.030213,95399.0
5591,40662.565066,60.704142,8.721363,5875.009428,271680.0
6322,144220.362234,75.760286,9.091234,4537.786246,0.0
7053,116541.544767,28.780066,3.453608,2706.221559,443.0


In [36]:
ep8_zero_npy = ep8_zero.values
ep8_zero_npy = ep8_zero_npy.reshape(8,5,5)
ep8_zero_npy[0, ]

array([[1.71968029e+05, 1.06518962e+03, 1.53035778e+02, 4.84267453e+04,
        0.00000000e+00],
       [2.10510226e+05, 4.04647729e+02, 5.81357335e+01, 2.58629901e+04,
        1.47700000e+03],
       [2.30974807e+05, 3.22479310e+02, 4.63305980e+01, 2.92303835e+04,
        1.21000000e+04],
       [2.74604746e+05, 2.83129238e+02, 4.06771736e+01, 2.25194480e+04,
        1.87250000e+04],
       [2.40229218e+05, 2.45802521e+02, 3.53144447e+01, 1.74426647e+04,
        2.80350000e+04]])

In [40]:
N_g = 8 # number of age groups
N_p = 5 # number of ervas
N_t = 10

S_g = np.zeros((N_g,N_p,N_t))
I_g = np.zeros((N_g,N_p,N_t))
E_g = np.zeros((N_g,N_p,N_t))
R_g = np.zeros((N_g,N_p,N_t))
V_g = np.zeros((N_g,N_p,N_t))

In [41]:
for age_i in range(N_g):
    for erva_i in range(N_p):
        S_g[age_i, erva_i, 0] = ep8_zero_npy[age_i, erva_i, 0]/age_er[age_i, erva_i]
        I_g[age_i, erva_i, 0] = ep8_zero_npy[age_i, erva_i, 1]/age_er[age_i, erva_i]
        E_g[age_i, erva_i, 0] = ep8_zero_npy[age_i, erva_i, 2]/age_er[age_i, erva_i]
        R_g[age_i, erva_i, 0] = ep8_zero_npy[age_i, erva_i, 3]/age_er[age_i, erva_i]
        V_g[age_i, erva_i, 0] = ep8_zero_npy[age_i, erva_i, 4]/age_er[age_i, erva_i]

NameError: name 'age_er' is not defined

In [8]:
from fetch_data import fetch_hs_hospitalizations
from env_var import MAPPINGS, EPIDEMIC

In [12]:
hosp_by_erva = fetch_hs_hospitalizations(logger)
number_age_groups = 9
age_groups = MAPPINGS['age_groups'][number_age_groups]['names']
probs_age_icu = EPIDEMIC['p_c'][number_age_groups]
probs_age_ward = EPIDEMIC['p_H'][number_age_groups]
probs_age_death = EPIDEMIC['proportion_deaths_age'][number_age_groups]

hosp_by_erva_list = hosp_by_erva.values
dates = np.unique(hosp_by_erva_list[:, 0])
ervas = np.unique(hosp_by_erva_list[:, 1])

header = 'date;erva;age;ward;icu'
final_lines = [header, ]
for date in dates:
    for erva in ervas:
        values_day_erva = hosp_by_erva_list[np.where((hosp_by_erva_list[:, 0] == date) & (hosp_by_erva_list[:, 1] == erva))]
        ward_day_erva = 0 if len(values_day_erva[:, 3]) == 0 else values_day_erva[:, 3].item()
        icu_day_erva = 0 if len(values_day_erva[:, 4]) == 0 else values_day_erva[:, 3].item()
        death_day_erva = 0 if len(values_day_erva[:, 5]) == 0 else values_day_erva[:, 3].item()
        
        print(death_day_erva)

0
0
0
0
0
0
45
3
3
0
7
2
56
0
0
16
0
0
57
4
5
0
7
3
60
4
4
0
8
5
69
8
4
0
6
6
70
8
4
0
7
5
58
7
2
0
9
5
68
11
1
0
11
6
65
8
4
0
8
10
72
8
5
0
11
12
74
7
8
0
12
13
82
8
16
0
11
16
101
10
9
0
11
16
101
13
9
0
9
16
103
15
17
0
7
15
107
14
17
0
8
16
102
13
19
0
10
11
110
13
12
0
10
10
112
13
15
0
9
9
111
12
13
0
9
11
113
13
12
0
8
11
109
11
14
0
8
9
94
11
13
0
7
14
97
10
12
0
5
11
97
6
9
0
5
12
94
7
7
0
5
11
102
9
9
0
4
18
103
7
9
0
4
16
100
5
10
0
5
16
108
7
12
0
4
15
106
4
12
0
5
11
106
3
8
0
4
12
101
3
8
0
3
12
105
5
8
0
3
10
113
5
11
0
2
13
124
5
14
0
3
11
110
3
12
0
3
11
111
3
10
0
3
9
105
3
10
0
3
9
107
6
11
0
3
9
119
6
12
0
3
8
122
4
11
0
3
7
121
5
9
0
3
6
107
4
11
0
3
5
108
2
7
0
2
5
103
2
4
0
2
5
103
3
4
0
2
5
107
2
4
0
2
2
105
3
5
0
2
3
106
2
6
0
4
4
95
2
6
0
3
4
74
1
6
0
2
3
78
1
6
0
2
3
77
1
6
0
2
3
71
1
7
0
1
3
71
3
6
0
2
5
83
2
5
0
3
5
77
2
5
0
2
2
79
2
5
0
2
4
76
1
3
0
2
1
78
1
4
0
2
2
76
1
5
0
2
2
70
2
7
0
2
3
63
3
7
0
1
2
61
3
6
0
1
1
55
3
5
0
0
1
49
2
5
0
0
4
46
1
6
0
0
5

In [5]:
hosp_by_erva

,date,erva,hospitalized,ward,icu,dead
0,2020-03-21,HYKS,0,0,0,1
2,2020-03-25,HYKS,63,45,18,2
3,2020-03-25,KYS,6,3,3,0
4,2020-03-25,OYS,4,3,1,0
5,2020-03-25,TAYS,7,7,0,1
...,...,...,...,...,...,...
1493,2021-04-21,HYKS,86,64,22,585
1494,2021-04-21,KYS,7,1,6,81
1495,2021-04-21,OYS,12,11,1,56
1496,2021-04-21,TAYS,18,17,1,79
